# NBA DFS Backtest Runner

Run backtests using the DailyBacktest and BacktestRunner classes from src.backtest module.

**Data Source**: Local SQLite database at `nba_dfs.db`

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import logging
from datetime import datetime, timedelta

sys.path.append('..')

from src.backtest.backtest import DailyBacktest, BacktestRunner, WalkForwardValidator
from src.data.backtest_data_prep import BacktestDataPrep
from src.data.collectors.tank01_client import Tank01Client
from src.data.collectors.local_data_client import LocalDataClient
from dotenv import load_dotenv

load_dotenv()

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)


class MockStorage:
    """Mock storage for testing."""
    
    def __init__(self, base_path='../data'):
        self.base_path = base_path


print('Imports complete')

Imports complete


In [3]:
client = Tank01Client(api_key=os.getenv('TANK01_API_KEY'))
print('Client initialized')
client.get_dfs_salaries(date='20250120')

Client initialized


{'statusCode': 200,
 'body': {'date': '20250120',
  'draftkings': [{'pos': 'SG',
    'teamID': '9',
    'team': 'DET',
    'salary': '10300',
    'playerID': '94804285527',
    'longName': 'Cade Cunningham',
    'allValidPositions': ['PG']},
   {'pos': 'PF',
    'teamID': '2',
    'team': 'BOS',
    'salary': '10000',
    'playerID': '28628646399',
    'longName': 'Jayson Tatum',
    'allValidPositions': ['SF', 'PF']},
   {'pos': 'PG',
    'teamID': '1',
    'team': 'ATL',
    'salary': '9800',
    'playerID': '28978646789',
    'longName': 'Trae Young',
    'allValidPositions': ['PG']},
   {'pos': 'PF',
    'teamID': '20',
    'team': 'NY',
    'salary': '9700',
    'playerID': '28278119129',
    'longName': 'Karl-Anthony Towns',
    'allValidPositions': ['C']},
   {'pos': 'PF',
    'teamID': '24',
    'team': 'PHO',
    'salary': '9500',
    'playerID': '28336662792',
    'longName': 'Kevin Durant',
    'allValidPositions': ['PF']},
   {'pos': 'C',
    'teamID': '11',
    'team': 'HO

## Initialize Components

Initialize LocalDataClient to read from local database and storage handler.

In [10]:
client = LocalDataClient()
storage = MockStorage(base_path='../data')

print('LocalDataClient initialized')
print(f'Database: {client.db_path}')
print(f'Data directory: {client.data_dir}')
print(f'Storage base path: {storage.base_path}')

LocalDataClient initialized
Database: C:\Users\antho\OneDrive\Documents\Repositories\delapan-fantasy\nba_dfs.db
Data directory: C:\Users\antho\OneDrive\Documents\Repositories\delapan-fantasy\data
Storage base path: ../data


## Option 1: Daily Backtest

Run backtest for single slate with data preparation and feature generation.

In [11]:
data_prep = BacktestDataPrep(
    api_client=client
)

daily_backtest = DailyBacktest(
    data_prep=data_prep,
    storage=storage,
    seasons=['2024', '2025']
)

print('DailyBacktest initialized')

DailyBacktest initialized


### Run Single Date Backtest

In [12]:
game_date = '20250102'

result = daily_backtest.run_daily_backtest(
    game_date=game_date,
    model_fn=None,
    optimizer_fn=None
)

print('\n=== Backtest Result ===')
for key, value in result.items():
    if key not in ['projections', 'lineup']:
        print(f'{key}: {value}')

2025-10-05 05:32:36,302 - src.backtest.backtest - INFO - Starting daily backtest for 20250102
2025-10-05 05:32:36,303 - src.backtest.backtest - INFO - Preparing slate data for 20250102
2025-10-05 05:32:36,310 - src.backtest.backtest - INFO - Data preparation complete: 0 players found
2025-10-05 05:32:36,310 - src.backtest.backtest - INFO - Verifying data completeness for 20250102
2025-10-05 05:32:36,312 - src.backtest.backtest - INFO - Verification complete: 0/0 players have data
2025-10-05 05:32:36,312 - src.backtest.backtest - INFO - Processing 0 players
2025-10-05 05:32:36,312 - src.backtest.backtest - INFO - Preparing features for 0 players
2025-10-05 05:32:36,313 - src.backtest.backtest - INFO - Feature preparation complete: 0 with features, 0 without
2025-10-05 05:32:36,313 - src.backtest.backtest - INFO - Backtest complete for 20250102



=== Running Backtest for 20250102 ===

Preparing data for 20250102
Seasons to collect: ['2024', '2025']
Teams playing: 0 teams, 0 total (home+away)
Total players: 0
Players with existing data: 0
Players needing data collection: 0

Prepared features for 0 players

=== Backtest Result ===
game_date: 20250102
teams: []
total_players: 0
players_with_data: 0
players_with_features: 0
data_complete: True


### Run Multi-Date Backtest

In [ ]:
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 1, 7)

game_dates = []
current_date = start_date
while current_date <= end_date:
    game_dates.append(current_date.strftime('%Y%m%d'))
    current_date += timedelta(days=1)

print(f'Running backtest for {len(game_dates)} dates: {game_dates[0]} to {game_dates[-1]}')

results_df = daily_backtest.run_multi_date_backtest(
    game_dates=game_dates,
    model_fn=None,
    optimizer_fn=None
)

print('\n=== Multi-Date Backtest Results ===')
print(results_df)

### View Summary Statistics

In [ ]:
summary = daily_backtest.get_summary()

print('\n=== Summary Statistics ===')
for key, value in summary.items():
    print(f'{key}: {value}')

## Option 2: Walk-Forward Validation Backtest

Run walk-forward validation with train/test splits.

In [ ]:
def dummy_model_fn():
    from sklearn.ensemble import RandomForestRegressor
    return RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)

validator = WalkForwardValidator(
    train_window=30,
    test_window=1,
    step_size=1
)

backtest_runner = BacktestRunner(
    model_fn=dummy_model_fn,
    storage=storage,
    validator=validator
)

print('BacktestRunner initialized with WalkForwardValidator')

BacktestRunner initialized with WalkForwardValidator


### Define Feature Columns

In [ ]:
feature_columns = [
    'mins', 'pts', 'reb', 'ast', 'stl', 'blk', 'TOV',
    'fga', 'fgm', 'FGP', 'tpa', 'tpm', 'TPP',
    'fta', 'ftm', 'FTP', 'PF'
]

target_column = 'fpts'

print(f'Feature columns: {len(feature_columns)}')
print(f'Target column: {target_column}')

Feature columns: 17
Target column: fpts


### Run Walk-Forward Backtest

In [ ]:
backtest_start = '20241201'
backtest_end = '20241231'

print(f'Running walk-forward backtest from {backtest_start} to {backtest_end}')

backtest_results = backtest_runner.run(
    start_date=backtest_start,
    end_date=backtest_end,
    feature_columns=feature_columns,
    target_column=target_column
)

print('\n=== Walk-Forward Backtest Results ===')
for key, value in backtest_results.items():
    print(f'{key}: {value}')

### View Detailed Fold Results

In [ ]:
fold_results = backtest_runner.get_results()

print('\n=== Fold-by-Fold Results ===')
print(fold_results[['fold', 'train_start', 'train_end', 'test_start', 'test_end', 'mape', 'rmse', 'correlation']])

### Plot MAPE Over Time

In [ ]:
import matplotlib.pyplot as plt

if not fold_results.empty:
    plt.figure(figsize=(12, 6))
    plt.plot(fold_results['fold'], fold_results['mape'], marker='o', label='MAPE')
    plt.axhline(y=30, color='r', linestyle='--', label='30% Target')
    plt.xlabel('Fold')
    plt.ylabel('MAPE')
    plt.title('Model MAPE Across Walk-Forward Folds')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print('No fold results to plot')

## Data Usage Summary

Summary of database queries executed during backtest.

In [ ]:
print(f'\nTotal data queries: {client.get_request_count()}')
print(f'Data source: Local database at {client.db_path}')